# Predicting Match Winners with XGBoost Classifier

In this notebook, we aim to predict the winner of a match using an XGBoost classifier. We want to give higher importance to correctly predicting upsets, defined as instances where a player with a lower default ELO wins the match.

## Table of Contents
1. [Data Preparation](#data-preparation)
2. [Baseline Model](#baseline-model)
3. [Weighted Model](#weighted-model)
4. [Hyperparameter Optimization with Optuna](#hyperparameter-optimization)
5. [Conclusion](#conclusion)


In [108]:
# Standard library imports
import datetime
import os
from collections import deque
import time

# Third-party imports
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns

from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import accuracy_score, log_loss
import optuna

from tqdm import tqdm

if os.path.exists('/workspace/data_2'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data_2/'
else:
    data_path = '../data/'
    
# if torch.cuda.is_available() == False:
#     RuntimeError("GPU detected: False")
#     print("GPU detected: False")
# else:
#     device = torch.device("cuda")
#     print("The GPU is detected.")



### Load Data

In [52]:
dataset_df = pd.read_pickle(data_path + 'full_dataset_df.pkl')


Identify columns for training.

In [53]:
for i, col in enumerate(dataset_df.columns):
    print(i, col)
    

0 key_x
1 game
2 tournament_key
3 winner_id
4 loser_id
5 p1_id
6 p2_id
7 p1_score
8 p2_score
9 valid_score
10 best_of
11 location_names
12 bracket_name
13 bracket_order
14 set_order
15 game_data
16 top_8
17 top_8_location_names
18 valid_top_8_bracket
19 top_8_bracket_location_names
20 major
21 key_y
22 start
23 end
24 start_week
25 p1_characters
26 p2_characters
27 p1_consistent
28 p2_consistent
29 matchup_strings
30 end_week
31 players_have_history
32 (p1/p2)_sorted
33 (p1/p2)_was_sorted
34 results_sorted
35 results
36 matchup_1
37 matchup_2
38 matchup_3
39 matchup_4
40 matchup_5
41 matchup_6
42 matchup_7
43 matchup_8
44 matchup_9
45 matchup_10
46 winner
47 p1_default_elo
48 p2_default_elo
49 p1_default_rd
50 p2_default_rd
51 p1_default_updates
52 p2_default_updates
53 start_index
54 start_date
55 p1_fox_count
56 p1_falco_count
57 p1_marth_count
58 p1_sheik_count
59 p1_captainfalcon_count
60 p1_jigglypuff_count
61 p1_peach_count
62 p1_luigi_count
63 p1_samus_count
64 p1_ganondorf_coun

Separate the features of player one and player two

In [54]:
# Define features and target
features = (
    list(dataset_df.columns[36:46]) +
    list(dataset_df.columns[47:53]) +
    list(dataset_df.columns[55:])
).copy()
target = 'winner'

print(features)



['matchup_1', 'matchup_2', 'matchup_3', 'matchup_4', 'matchup_5', 'matchup_6', 'matchup_7', 'matchup_8', 'matchup_9', 'matchup_10', 'p1_default_elo', 'p2_default_elo', 'p1_default_rd', 'p2_default_rd', 'p1_default_updates', 'p2_default_updates', 'p1_fox_count', 'p1_falco_count', 'p1_marth_count', 'p1_sheik_count', 'p1_captainfalcon_count', 'p1_jigglypuff_count', 'p1_peach_count', 'p1_luigi_count', 'p1_samus_count', 'p1_ganondorf_count', 'p1_iceclimbers_count', 'p1_drmario_count', 'p1_yoshi_count', 'p1_pikachu_count', 'p1_link_count', 'p1_mrgameandwatch_count', 'p1_donkeykong_count', 'p1_mario_count', 'p1_zelda_count', 'p1_roy_count', 'p1_younglink_count', 'p1_kirby_count', 'p1_ness_count', 'p1_bowser_count', 'p1_pichu_count', 'p1_random_count', 'p1_mewtwo_count', 'p2_fox_count', 'p2_falco_count', 'p2_marth_count', 'p2_sheik_count', 'p2_captainfalcon_count', 'p2_jigglypuff_count', 'p2_peach_count', 'p2_luigi_count', 'p2_samus_count', 'p2_ganondorf_count', 'p2_iceclimbers_count', 'p2_drm

# Data Preparation


In [87]:
# 1. Define the 'expected_winner' column
# An expected winner is defined only when ELOs are strictly different
# If ELOs are equal, expected_winner is set to NaN
dataset_df['expected_winner'] = np.where(
    dataset_df['p1_default_elo'] > dataset_df['p2_default_elo'], 1,
    np.where(dataset_df['p1_default_elo'] < dataset_df['p2_default_elo'], 0, np.nan)
)

# 2. Define 'upset' only when 'expected_winner' is not NaN
dataset_df['upset'] = np.where(
    dataset_df['expected_winner'].notna() & (dataset_df['winner'] != dataset_df['expected_winner']), 1, 0
)

# 3. Remove matches where ELOs are equal (expected_winner is NaN)
dataset_df = dataset_df[dataset_df['expected_winner'].notna()].reset_index(drop=True)

# 4. Split the data into upsets and non-upsets
upsets_df = dataset_df[dataset_df['upset'] == 1]
non_upsets_df = dataset_df[dataset_df['upset'] == 0]

# 5. Split each into train and test sets
upsets_train, upsets_test = train_test_split(
    upsets_df, test_size=0.2, random_state=42
)
non_upsets_train, non_upsets_test = train_test_split(
    non_upsets_df, test_size=0.2, random_state=42
)

# 6. Combine the train and test sets
train_data = pd.concat([upsets_train, non_upsets_train]).sample(
    frac=1, random_state=42
).reset_index(drop=True)
test_data = pd.concat([upsets_test, non_upsets_test]).sample(
    frac=1, random_state=42
).reset_index(drop=True)

# 7. Separate features and target
X_train = train_data[features]
y_train = train_data[target]

X_test = test_data[features]
y_test = test_data[target]


In [88]:
X_train.columns

Index(['matchup_1', 'matchup_2', 'matchup_3', 'matchup_4', 'matchup_5',
       'matchup_6', 'matchup_7', 'matchup_8', 'matchup_9', 'matchup_10',
       'p1_default_elo', 'p2_default_elo', 'p1_default_rd', 'p2_default_rd',
       'p1_default_updates', 'p2_default_updates', 'p1_fox_count',
       'p1_falco_count', 'p1_marth_count', 'p1_sheik_count',
       'p1_captainfalcon_count', 'p1_jigglypuff_count', 'p1_peach_count',
       'p1_luigi_count', 'p1_samus_count', 'p1_ganondorf_count',
       'p1_iceclimbers_count', 'p1_drmario_count', 'p1_yoshi_count',
       'p1_pikachu_count', 'p1_link_count', 'p1_mrgameandwatch_count',
       'p1_donkeykong_count', 'p1_mario_count', 'p1_zelda_count',
       'p1_roy_count', 'p1_younglink_count', 'p1_kirby_count', 'p1_ness_count',
       'p1_bowser_count', 'p1_pichu_count', 'p1_random_count',
       'p1_mewtwo_count', 'p2_fox_count', 'p2_falco_count', 'p2_marth_count',
       'p2_sheik_count', 'p2_captainfalcon_count', 'p2_jigglypuff_count',
       'p2

In [89]:
print(dataset_df['upset'].sum()/dataset_df.shape[0])

0.2546142541139352


# Baseline Model


In [131]:
# Train XGBoost classifier without weights
baseline_model = xgb.XGBClassifier( eval_metric='error')
baseline_model.fit(X_train, y_train)

# Predict on test data
y_pred_baseline = baseline_model.predict(X_test)

# Overall accuracy
overall_accuracy = accuracy_score(y_test, y_pred_baseline)

# Add predictions to test_data
test_data['prediction'] = y_pred_baseline

# Compute accuracies for upsets and non-upsets
upset_mask = test_data['upset'] == 1
accuracy_upsets = accuracy_score(
    y_test[upset_mask], y_pred_baseline[upset_mask]
)
accuracy_non_upsets = accuracy_score(
    y_test[~upset_mask], y_pred_baseline[~upset_mask]
)

print(f"Baseline Model Overall Accuracy: {overall_accuracy:.4f}")
print(f"Baseline Model Upset Accuracy: {accuracy_upsets:.4f}")
print(f"Baseline Model Non-Upset Accuracy: {accuracy_non_upsets:.4f}")





Baseline Model Overall Accuracy: 0.7763
Baseline Model Upset Accuracy: 0.3059
Baseline Model Non-Upset Accuracy: 0.9369


## Baseline Model

We train a baseline XGBoost classifier without any weighting to see how well it predicts both upsets and non-upsets.

- **Overall Accuracy**: Measures the model's performance across all matches.
- **Upset Accuracy**: How often the model correctly predicts upsets.
- **Non-Upset Accuracy**: How often the model correctly predicts matches without upsets.

The baseline accuracies are:

- **Overall Accuracy**: 0.7763
- **Upset Accuracy**: 0.3059
- **Non-Upset Accuracy**: 0.9369


# Weighted Model


In [96]:
# Create sample weights for the training data
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 3.5  # Upsets get a weight of 3.5

# Train XGBoost classifier with sample weights
weighted_model = xgb.XGBClassifier( eval_metric='logloss')
weighted_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_weighted = weighted_model.predict(X_test)

# Overall accuracy
overall_accuracy_weighted = accuracy_score(y_test, y_pred_weighted)

# Add predictions to test_data
test_data['prediction_weighted'] = y_pred_weighted

# Compute accuracies for upsets and non-upsets
accuracy_upsets_weighted = accuracy_score(
    y_test[upset_mask], y_pred_weighted[upset_mask]
)
accuracy_non_upsets_weighted = accuracy_score(
    y_test[~upset_mask], y_pred_weighted[~upset_mask]
)

print(f"Weighted Model Overall Accuracy: {overall_accuracy_weighted:.4f}")
print(f"Weighted Model Upset Accuracy: {accuracy_upsets_weighted:.4f}")
print(f"Weighted Model Non-Upset Accuracy: {accuracy_non_upsets_weighted:.4f}")


Weighted Model Overall Accuracy: 0.6830
Weighted Model Upset Accuracy: 0.6912
Weighted Model Non-Upset Accuracy: 0.6802


## Weighted Model

To improve the model's ability to predict upsets, we assign a higher weight (2x) to upset instances in the loss function during training.

The weighted model accuracies are:

- **Overall Accuracy**: 0.7619
- **Upset Accuracy**: 0.4726
- **Non-Upset Accuracy**: 0.8607

Comparing these results with the baseline model helps us understand the impact of weighting on model performance.


In [80]:
# Function to optimize upset weight using Optuna
def objective(trial):
    # Suggest a weight between 1 and 5
    upset_weight = trial.suggest_float('upset_weight', 0.8, 1.2)
    
    # Create sample weights
    sample_weight = np.ones(len(y_train))
    sample_weight[train_data['upset'] == 1] = upset_weight
    
    # Train model
    model = xgb.XGBClassifier( eval_metric='logloss')
    model.fit(X_train, y_train, sample_weight=sample_weight)
    
    # Predict on test data
    y_pred = model.predict(X_test)
    
    # Calculate overall accuracy
    accuracy = accuracy_score(y_test, y_pred)
    
    return accuracy  # Optuna minimizes the objective

# Create a study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50, show_progress_bar=True, timeout=600)

# Best upset weight
best_upset_weight = study.best_params['upset_weight']
print(f"Best Upset Weight: {best_upset_weight:.4f}")

# Train final model with the best upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = best_upset_weight

final_model = xgb.XGBClassifier(eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")


[I 2024-11-27 18:23:11,137] A new study created in memory with name: no-name-89c6227f-d9b1-4fdb-bee7-fad24905c877


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2024-11-27 18:23:19,144] Trial 0 finished with value: 0.7763939044978366 and parameters: {'upset_weight': 0.9015182568659634}. Best is trial 0 with value: 0.7763939044978366.
[I 2024-11-27 18:23:32,754] Trial 1 finished with value: 0.7748398421340966 and parameters: {'upset_weight': 0.7195488286093267}. Best is trial 0 with value: 0.7763939044978366.
[I 2024-11-27 18:23:39,684] Trial 2 finished with value: 0.7690009279652963 and parameters: {'upset_weight': 0.5408382256180367}. Best is trial 0 with value: 0.7763939044978366.
[I 2024-11-27 18:23:50,259] Trial 3 finished with value: 0.7749236944199099 and parameters: {'upset_weight': 0.7579044366072831}. Best is trial 0 with value: 0.7763939044978366.
[I 2024-11-27 18:23:59,360] Trial 4 finished with value: 0.7756448240779044 and parameters: {'upset_weight': 1.1876425995058697}. Best is trial 0 with value: 0.7763939044978366.
[I 2024-11-27 18:24:11,957] Trial 5 finished with value: 0.7751333251344432 and parameters: {'upset_weight': 0

In [122]:
# Function to optimize hyperparameters using Optuna
def objective(trial):
    # Suggest hyperparameters for XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'n_jobs': -1,
        'random_state': 42,
    }
    
    upset_weight = 3
    
    # Create sample weights for training data
    sample_weight = np.ones(len(y_train))
    sample_weight[train_data['upset'] == 1] = upset_weight
    
    # Train model
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    
    # Predict probabilities on test data
    y_pred_proba = model.predict_proba(X_test)
    
    # Create sample weights for test data
    sample_weight_test = np.ones(len(y_test))
    sample_weight_test[test_data['upset'] == 1] = upset_weight
    
    # Calculate log loss
    loss = log_loss(y_test, y_pred_proba, sample_weight=sample_weight_test)
    
    # Optionally, print accuracy for monitoring
    accuracy = accuracy_score(y_test, model.predict(X_test))
    print(f"Trial {trial.number}: Log Loss = {loss:.5f}, Accuracy = {accuracy:.5%}")
    
    return loss  # Optuna will minimize this metric

# Create a study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=300, timeout=3600, show_progress_bar=True)

# Best hyperparameters
best_params = study.best_params
# best_upset_weight = best_params.pop('upset_weight')
# print(f"Best Upset Weight: {best_upset_weight:.4f}")
print("Best Hyperparameters:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 3

final_model = xgb.XGBClassifier(**best_params, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")


[I 2024-11-27 20:33:20,498] A new study created in memory with name: no-name-62347ecd-dc02-4a6b-b716-4eff476d12be


  0%|          | 0/300 [00:00<?, ?it/s]

Trial 0: Log Loss = 0.63623, Accuracy = 72.70832%
[I 2024-11-27 20:34:37,769] Trial 0 finished with value: 0.6362348842351547 and parameters: {'n_estimators': 433, 'max_depth': 15, 'learning_rate': 0.1672704480182783, 'min_child_weight': 9, 'gamma': 0.0028772981386906405, 'subsample': 0.6645397320322507, 'colsample_bytree': 0.7986937021243965, 'reg_alpha': 0.15497263510813206, 'reg_lambda': 0.02607990868191984}. Best is trial 0 with value: 0.6362348842351547.
Trial 1: Log Loss = 0.58967, Accuracy = 73.22652%
[I 2024-11-27 20:34:56,693] Trial 1 finished with value: 0.5896680557446415 and parameters: {'n_estimators': 424, 'max_depth': 6, 'learning_rate': 0.06821367025722622, 'min_child_weight': 7, 'gamma': 2.523286511401221e-07, 'subsample': 0.9653928452412004, 'colsample_bytree': 0.9877056996992711, 'reg_alpha': 7.2462564594556784e-06, 'reg_lambda': 6.846985080569071e-06}. Best is trial 1 with value: 0.5896680557446415.
Trial 2: Log Loss = 0.58916, Accuracy = 73.63181%
[I 2024-11-27 20:

In [123]:
# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 2

final_model = xgb.XGBClassifier(**best_params, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")

Final Model Overall Accuracy: 0.7618
Final Model Upset Accuracy: 0.5089
Final Model Non-Upset Accuracy: 0.8481


In [124]:
# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 1.5

final_model = xgb.XGBClassifier(**best_params, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")

Final Model Overall Accuracy: 0.7738
Final Model Upset Accuracy: 0.4226
Final Model Non-Upset Accuracy: 0.8937


In [132]:
best = {'n_estimators': 332,
  'max_depth': 13,
  'learning_rate': 0.0329014414333458,
  'min_child_weight': 3,
  'gamma': 0.024318270664498532,
  'subsample': 0.8478652099231178,
  'colsample_bytree': 0.6737054254112979,
  'reg_alpha': 3.090492668111583e-05,
  'reg_lambda': 6.748516964647809e-06,
  'tree_method': 'hist'}

# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 1

final_model = xgb.XGBClassifier(**best, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")

Final Model Overall Accuracy: 0.7783
Final Model Upset Accuracy: 0.3237
Final Model Non-Upset Accuracy: 0.9336


In [116]:
print("Best Hyperparameters:")
best = {'n_estimators': 864,
        'max_depth': 3,
        'learning_rate': 0.01610478829322539,
        'min_child_weight': 8,
        'gamma': 5.003832819230665e-08,
        'subsample': 0.6645683689341554,
        'colsample_bytree': 0.6581693927090808,
        'reg_alpha': 9.956839465089495e-06,
        'reg_lambda': 0.0365646384855557,
        'tree_method': 'hist'}

# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = 1

final_model = xgb.XGBClassifier(**best, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")

Best Hyperparameters:
Final Model Overall Accuracy: 0.7750
Final Model Upset Accuracy: 0.3116
Final Model Non-Upset Accuracy: 0.9333


In [112]:
# Function to optimize hyperparameters using Optuna
def objective(trial):
    # Suggest hyperparameters for XGBoost
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3, log=True),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'gamma': trial.suggest_float('gamma', 1e-8, 1.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0, log=True),
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'n_jobs': -1,
        'random_state': 42,
    }
    
    # Suggest a weight for upsets between 0.8 and 1.2
    upset_weight = trial.suggest_float('upset_weight', 2.5, 3.5)
    
    # Create sample weights
    sample_weight = np.ones(len(y_train))
    sample_weight[train_data['upset'] == 1] = upset_weight
    
    # Train model
    model = xgb.XGBClassifier(**params)
    model.fit(X_train, y_train, sample_weight=sample_weight)
    
    # Predict on test data
    y_pred = model.predict(X_test)
    
    # Calculate overall accuracy
    loss = accuracy_score(y_test, y_pred)
    
    return loss  # Optuna will maximize this metric

# Create a study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300, timeout=3600*2, show_progress_bar=True)

# Best hyperparameters
best_params = study.best_params
best_upset_weight = best_params.pop('upset_weight')
print(f"Best Upset Weight: {best_upset_weight:.4f}")
print("Best Hyperparameters:")
for key, value in best_params.items():
    print(f"  {key}: {value}")

# Train final model with the best hyperparameters and upset weight
sample_weight = np.ones(len(y_train))
sample_weight[train_data['upset'] == 1] = best_upset_weight

final_model = xgb.XGBClassifier(**best_params, eval_metric='logloss')
final_model.fit(X_train, y_train, sample_weight=sample_weight)

# Predict on test data
y_pred_final = final_model.predict(X_test)

# Overall accuracy
overall_accuracy_final = accuracy_score(y_test, y_pred_final)

# Compute accuracies for upsets and non-upsets
accuracy_upsets_final = accuracy_score(
    y_test[upset_mask], y_pred_final[upset_mask]
)
accuracy_non_upsets_final = accuracy_score(
    y_test[~upset_mask], y_pred_final[~upset_mask]
)

print(f"Final Model Overall Accuracy: {overall_accuracy_final:.4f}")
print(f"Final Model Upset Accuracy: {accuracy_upsets_final:.4f}")
print(f"Final Model Non-Upset Accuracy: {accuracy_non_upsets_final:.4f}")


[I 2024-11-27 20:10:43,653] A new study created in memory with name: no-name-18223380-6276-4a0f-b3e5-b2d1020bd4cc


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2024-11-27 20:12:07,126] Trial 0 finished with value: 0.7280893977169818 and parameters: {'n_estimators': 299, 'max_depth': 14, 'learning_rate': 0.01729852492241346, 'min_child_weight': 9, 'gamma': 1.4529302114839138e-05, 'subsample': 0.6149531125392811, 'colsample_bytree': 0.7779283234663867, 'reg_alpha': 0.0006729737342550955, 'reg_lambda': 7.369018959922341e-06, 'upset_weight': 2.9296157385156523}. Best is trial 0 with value: 0.7280893977169818.
[W 2024-11-27 20:12:29,641] Trial 1 failed with parameters: {'n_estimators': 975, 'max_depth': 6, 'learning_rate': 0.14536026601462412, 'min_child_weight': 1, 'gamma': 0.0003386691737871086, 'subsample': 0.9687918033792976, 'colsample_bytree': 0.8355601056224924, 'reg_alpha': 3.174130591698756e-06, 'reg_lambda': 3.338061874586353e-05, 'upset_weight': 2.6317921749361326} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/optuna/study/_optimize.py", line 19

KeyboardInterrupt: 